In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from keras_preprocessing.image import ImageDataGenerator
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.layers import Dense, Concatenate
from keras.models import load_model
from tensorflow.keras.layers import Input,Dense, GlobalAveragePooling2D 
from tensorflow.keras.models import Model
BATCH_SIZE =32

In [3]:
def append_ext(fn):
    if not '.jpg' in str(fn):
        return str(fn) + ".jpg"
    else:
        return str(fn)
    
def gen():
    for line in poselist:
        print( len((line[1:])))
        yield line[1:]

class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, list_pose, batch_size= BATCH_SIZE, dim=(32,32,32), n_channels= 1, shuffle= True):
        self.dim = dim
        self.batch_size = batch_size
        self.list_pose = list_pose
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()
        self.n = 0
    def NormalizeData(self,data):
        return (data - np.min(data)) / (np.max(data) - np.min(data))
    
    def __len__(self):
        'Denotes the number of batches per epoch'
        #print("__len__ ",int(np.floor(len(self.list_pose) / self.batch_size)))
        return int(959)

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_pose_temp = [self.list_pose[k] for k in indexes]

        X = self.__data_generation(list_pose_temp)

        return X

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_pose))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __next__(self):
        if self.n >= self.max:
            self.n = 0
        result = self.__getitem__(self.n)
        self.n += 1
        return result

    def __data_generation(self, list_pose_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        #print("__datagen__ ", len(X), len(X[0]))

        for i, each in enumerate(list_pose_temp):
            # Store sample
            
            print("each in gen ", each)
            X[i,] =np.reshape(np.load('C:/Users/IDU/OneDrive - GTÜ/Desktop/TEZ/poses/NPdata/'+ each ,allow_pickle=True),(66,1))
        print(len(X)," length x")
        # Generate data
        ##for i, pose in enumerate(list_pose_temp):
            # Store sample
        
         ##   X[i,] =np.reshape( self.NormalizeData(np.array(pose[1:])),(66,1) )   
        #y= np.zeros((64,64))
        
        #arr = X[:length-19,:]
        return X
    

In [4]:
prod_df = pd.read_csv('C:/Users/IDU/OneDrive - GTÜ/Desktop/TEZ/MODANISA/prod_details.csv', low_memory=False)

prod_df = prod_df[['productid', 'category']]

df = prod_df.loc[ prod_df['category'].isin(['Tesettür Elbise','Takım','Tunik'])]

num_classes = 3

df["productid"] = df["productid"].apply(append_ext) #append jpg extentions


df = df.astype('str')


C:\Users\IDU\AppData\Local\Temp/ipykernel_11120/3044359244.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["productid"] = df["productid"].apply(append_ext) #append jpg extentions


In [5]:

params = {'dim': (66,),
          'batch_size': BATCH_SIZE,
          
          'n_channels': 1,""
          'shuffle': False}

"""path = "C:/Users/IDU/OneDrive - GTÜ/Desktop/TEZ/poses/NPdata/"
dir_list = os.listdir(path)
poselist =[]
for i, each in enumerate(dir_list):
    print("each ",each)
    if (each.split("."))[0] in df.values:
        print(each," appended")
        poselist.append(each)"""
poselist = np.load("C:/Users/IDU/OneDrive - GTÜ/Desktop/TEZ/poses/poselist.npy", allow_pickle=True )        
#print("num of poses ", len(poselist))

In [6]:
#np.save(r"C:\Users\IDU\OneDrive - GTÜ\Desktop\TEZ\poses\poselist.npy", np.array(poselist), allow_pickle=True)

In [7]:
len(poselist)

31393

In [8]:
# Generators
pose_generator = DataGenerator(poselist, **params)

In [9]:
#print("pose ",tf.shape(pose_generator))
input_shape=(100,135,3)

model = ResNet50( include_top= False , input_shape= input_shape , classes= num_classes)

#pose_generator = tf.constant(pose_generator, dtype=tf.float32,shape=(981,13,66))

#pose_generator = tf.keras.layers.Flatten()(pose_generator)
#pose_generator = tf.cast(pose_generator, dtype = tf.float32)

#reshape_layer = tf.keras.layers.Reshape((960,32,66,))(pose_generator)

#x = tf.keras.layers.GlobalAveragePooling2D()(model.output)
x = tf.keras.layers.Flatten()(model.output)

print(tf.shape(model.output))
x = Dense(66, activation='relu')(x)
#x= tf.expand_dims(x,axis= 0)
#x= tf.expand_dims(x,axis= -1)
#x = tf.tile(model.output, multiples=[1,8,1,1])
#x = tf.keras.layers.Reshape((4,5,66,1))(x)

print(tf.shape(x))

concat = tf.concat([x, pose_generator],axis=0 )

x = Dense(512, activation='relu')(concat)

y = Dense(num_classes, activation= 'softmax')(x)

#y = Dense(num_classes, activation= 'relu')(x)

model = tf.keras.models.Model(inputs= model.input, outputs= y)

Tensor("Shape:0", shape=(4,), dtype=int32)
Tensor("Shape_1:0", shape=(2,), dtype=int32)
each in gen  1001277.npy
each in gen  1001279.npy
each in gen  1001317.npy
each in gen  1001804.npy
each in gen  1001814.npy
each in gen  1001815.npy
each in gen  1001816.npy
each in gen  1001817.npy
each in gen  1001818.npy
each in gen  1002403.npy
each in gen  1002417.npy
each in gen  1002418.npy
each in gen  1002419.npy
each in gen  1002420.npy
each in gen  1002421.npy
each in gen  1002422.npy
each in gen  1002423.npy
each in gen  1002424.npy
each in gen  1002431.npy
each in gen  100389.npy
each in gen  100390.npy
each in gen  1006098.npy
each in gen  1006099.npy
each in gen  1006100.npy
each in gen  1006101.npy
each in gen  1006102.npy
each in gen  1006103.npy
each in gen  1006104.npy
each in gen  1006105.npy
each in gen  1006106.npy
each in gen  1006108.npy
each in gen  1006109.npy
32  length x
each in gen  1001277.npy
each in gen  1001279.npy
each in gen  1001317.npy
each in gen  1001804.npy
e

each in gen  1054981.npy
each in gen  1054982.npy
each in gen  1055008.npy
each in gen  1055028.npy
each in gen  1055029.npy
each in gen  1055030.npy
each in gen  1055032.npy
each in gen  1055033.npy
each in gen  1055034.npy
each in gen  1055036.npy
each in gen  1055037.npy
each in gen  1055288.npy
each in gen  1055289.npy
each in gen  1055290.npy
32  length x
each in gen  1055291.npy
each in gen  1055342.npy
each in gen  1055343.npy
each in gen  1055344.npy
each in gen  1055345.npy
each in gen  1055473.npy
each in gen  1055474.npy
each in gen  1055475.npy
each in gen  1055735.npy
each in gen  1055736.npy
each in gen  1055737.npy
each in gen  1056446.npy
each in gen  1056447.npy
each in gen  1056685.npy
each in gen  1056686.npy
each in gen  1056687.npy
each in gen  1056689.npy
each in gen  1056690.npy
each in gen  1056691.npy
each in gen  1056692.npy
each in gen  1056693.npy
each in gen  1056694.npy
each in gen  1056695.npy
each in gen  1056696.npy
each in gen  1056697.npy
each in gen 

each in gen  1076798.npy
each in gen  1076799.npy
each in gen  1076800.npy
each in gen  1076801.npy
each in gen  1076802.npy
each in gen  1076837.npy
each in gen  1076838.npy
each in gen  1076839.npy
each in gen  1076840.npy
each in gen  1076841.npy
each in gen  1076901.npy
each in gen  1076909.npy
each in gen  1077886.npy
each in gen  1077887.npy
each in gen  1077907.npy
each in gen  1077908.npy
each in gen  1079106.npy
each in gen  1079108.npy
each in gen  1079356.npy
each in gen  1079373.npy
each in gen  1079503.npy
each in gen  1079509.npy
each in gen  1079670.npy
each in gen  1079680.npy
each in gen  1079682.npy
each in gen  1079683.npy
each in gen  1079685.npy
each in gen  1079693.npy
each in gen  1079694.npy
each in gen  1079695.npy
each in gen  1079696.npy
32  length x
each in gen  1079697.npy
each in gen  1079698.npy
each in gen  1079699.npy
each in gen  1079700.npy
each in gen  1079701.npy
each in gen  1079702.npy
each in gen  1079703.npy
each in gen  1079704.npy
each in gen 

each in gen  1049455.npy
each in gen  1049456.npy
each in gen  1049457.npy
each in gen  1049458.npy
each in gen  1049461.npy
each in gen  1049462.npy
each in gen  1049463.npy
each in gen  1049464.npy
each in gen  1049465.npy
each in gen  1049466.npy
each in gen  1049467.npy
each in gen  1049468.npy
each in gen  1049469.npy
each in gen  1049473.npy
each in gen  1049474.npy
each in gen  1049478.npy
each in gen  1049479.npy
each in gen  1049480.npy
32  length x
each in gen  1049481.npy
each in gen  1049482.npy
each in gen  1049483.npy
each in gen  1049484.npy
each in gen  1049485.npy
each in gen  1049486.npy
each in gen  1049487.npy
each in gen  1049488.npy
each in gen  1049598.npy
each in gen  1050411.npy
each in gen  1050430.npy
each in gen  1050432.npy
each in gen  1050726.npy
each in gen  1050727.npy
each in gen  1050742.npy
each in gen  1050743.npy
each in gen  1050746.npy
each in gen  1050777.npy
each in gen  1050778.npy
each in gen  1050779.npy
each in gen  1050780.npy
each in gen 

each in gen  1070883.npy
each in gen  1070884.npy
each in gen  1070885.npy
each in gen  1070886.npy
each in gen  1070887.npy
each in gen  1070888.npy
each in gen  1070889.npy
each in gen  1070890.npy
each in gen  1070891.npy
each in gen  1070892.npy
each in gen  1070893.npy
each in gen  1071151.npy
each in gen  1071201.npy
each in gen  1071202.npy
each in gen  1071203.npy
each in gen  1071204.npy
each in gen  1071205.npy
each in gen  1071206.npy
each in gen  1071207.npy
each in gen  1071208.npy
each in gen  1071209.npy
each in gen  1071210.npy
each in gen  1071211.npy
32  length x
each in gen  1071212.npy
each in gen  1071213.npy
each in gen  1071214.npy
each in gen  1071215.npy
each in gen  1071216.npy
each in gen  1071217.npy
each in gen  1071218.npy
each in gen  1071219.npy
each in gen  1071220.npy
each in gen  1071221.npy
each in gen  1071225.npy
each in gen  1071226.npy
each in gen  1071227.npy
each in gen  1071686.npy
each in gen  1071687.npy
each in gen  1072818.npy
each in gen 

...

each in gen  8085063.npy
each in gen  8085064.npy
each in gen  8085065.npy
each in gen  8085066.npy
each in gen  8085067.npy
each in gen  8085068.npy
each in gen  8085069.npy
32  length x
each in gen  8085071.npy
each in gen  8085072.npy
each in gen  8085073.npy
each in gen  8085247.npy
each in gen  8085248.npy
each in gen  8085249.npy
each in gen  8085250.npy
each in gen  8085251.npy
each in gen  8085252.npy
each in gen  8085253.npy
each in gen  8085659.npy
each in gen  8085660.npy
each in gen  8085661.npy
each in gen  8085662.npy
each in gen  8085665.npy
each in gen  8085666.npy
each in gen  8085667.npy
each in gen  8085670.npy
each in gen  8085671.npy
each in gen  8085672.npy
each in gen  8085673.npy
each in gen  8085674.npy
each in gen  8085675.npy
each in gen  8085676.npy
each in gen  8085678.npy
each in gen  8085679.npy
each in gen  8085680.npy
each in gen  8085683.npy
each in gen  8085684.npy
each in gen  8085685.npy
each in gen  8085686.npy
each in gen  8085687.npy
32  length x

each in gen  8093619.npy
each in gen  8093621.npy
each in gen  8093625.npy
each in gen  8093628.npy
each in gen  8093635.npy
each in gen  8093638.npy
each in gen  8093639.npy
each in gen  8093640.npy
each in gen  8093643.npy
each in gen  8093644.npy
each in gen  8093645.npy
each in gen  8093646.npy
each in gen  8093653.npy
each in gen  8093655.npy
each in gen  8093658.npy
each in gen  8093661.npy
each in gen  8093662.npy
each in gen  8093663.npy
32  length x
each in gen  8093668.npy
each in gen  8093670.npy
each in gen  8093674.npy
each in gen  8093675.npy
each in gen  8093676.npy
each in gen  8093680.npy
each in gen  8093682.npy
each in gen  8093685.npy
each in gen  8093686.npy
each in gen  8093689.npy
each in gen  8093840.npy
each in gen  8093844.npy
each in gen  8093855.npy
each in gen  8093856.npy
each in gen  8093857.npy
each in gen  8093860.npy
each in gen  8093863.npy
each in gen  8093866.npy
each in gen  8093885.npy
each in gen  8093886.npy
each in gen  8093887.npy
each in gen 

each in gen  8104827.npy
32  length x
each in gen  8104834.npy
each in gen  811613.npy
each in gen  811891.npy
each in gen  811892.npy
each in gen  811893.npy
each in gen  811894.npy
each in gen  811895.npy
each in gen  811896.npy
each in gen  811897.npy
each in gen  811898.npy
each in gen  811899.npy
each in gen  811900.npy
each in gen  811901.npy
each in gen  811902.npy
each in gen  811903.npy
each in gen  811904.npy
each in gen  811905.npy
each in gen  811906.npy
each in gen  811907.npy
each in gen  811908.npy
each in gen  811909.npy
each in gen  811910.npy
each in gen  811911.npy
each in gen  811912.npy
each in gen  811913.npy
each in gen  812467.npy
each in gen  812890.npy
each in gen  812891.npy
each in gen  812892.npy
each in gen  812895.npy
each in gen  812896.npy
each in gen  812897.npy
32  length x
each in gen  812898.npy
each in gen  812899.npy
each in gen  812900.npy
each in gen  812904.npy
each in gen  812905.npy
each in gen  812906.npy
each in gen  812907.npy
each in gen 

each in gen  855247.npy
each in gen  855248.npy
each in gen  857415.npy
each in gen  857429.npy
each in gen  858272.npy
32  length x
each in gen  858273.npy
each in gen  858274.npy
each in gen  858275.npy
each in gen  858276.npy
each in gen  858277.npy
each in gen  858278.npy
each in gen  858279.npy
each in gen  858280.npy
each in gen  858281.npy
each in gen  858282.npy
each in gen  858283.npy
each in gen  858284.npy
each in gen  858285.npy
each in gen  858286.npy
each in gen  858287.npy
each in gen  858288.npy
each in gen  858289.npy
each in gen  858300.npy
each in gen  858301.npy
each in gen  858302.npy
each in gen  858303.npy
each in gen  858304.npy
each in gen  858305.npy
each in gen  858306.npy
each in gen  858307.npy
each in gen  858308.npy
each in gen  858309.npy
each in gen  858310.npy
each in gen  858311.npy
each in gen  858312.npy
each in gen  858313.npy
each in gen  858314.npy
32  length x
each in gen  858315.npy
each in gen  858318.npy
each in gen  858319.npy
each in gen  8

ValueError: Can't convert non-rectangular Python sequence to Tensor.

In [10]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 135, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 106, 141, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 50, 68, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 50, 68, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [11]:
df.category = pd.Categorical(df.category)

df['category'] = df.category.cat.codes



df = df.astype('str')


In [12]:
df.head()

,productid,category
3,1168509.jpg,2
5,7981329.jpg,0
20,7978162.jpg,1
44,1792618.jpg,1
64,1371598.jpg,2


In [ ]:
model.compile( optimizer= "adam", loss="categorical_crossentropy", metrics= ['accuracy'] )


datagen = ImageDataGenerator(rescale=1./255, 
                             validation_split=0.2 )

train_generator= datagen.flow_from_dataframe(df, 
                                            directory= 'C:/Users/IDU/Desktop/dataset/',
                                            x_col= 'productid',
                                            y_col= 'category',
                                            subset= "training",
                                            color_mode= 'rgb',
                                            class_mode = 'categorical',
                                            target_size= (100,135),
                                            batch_size= BATCH_SIZE )

test_generator= datagen.flow_from_dataframe(df,
                                            directory= 'C:/Users/IDU/Desktop/dataset/',
                                            subset= "validation",
                                            x_col= 'productid',
                                            y_col= 'category',
                                            class_mode="categorical",
                                            color_mode= 'rgb',
                                            target_size= (100,135),
                                            batch_size= BATCH_SIZE)


In [ ]:
66*32*960

In [ ]:
history= model.fit(train_generator ,
          validation_data= test_generator ,
          batch_size=BATCH_SIZE,
          epochs= 100)

In [21]:
ls =list(train_generator.next())
print(len(ls))
print(len(ls[0]))
print(len(ls[0][0]))
print(len(ls[0][0][0]))
#(2,32,100,135)

2
32
100
135


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
plt.savefig('model_accuracy.png')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
plt.savefig('model_loss.png')

In [ ]:
model.save('model_poseResnet.h5')

In [ ]:
print(len(poselist))
len(poselist[0])

In [ ]:
y = [[[1],[1],[1]],[[2],[2],[1]]],[[[1],[1],[1]],[[2],[2],[1]]]

In [ ]:
tf.shape(y)

In [ ]:
981/32

In [ ]:
30*32

In [ ]:

# pose_np = np.load("C:/Users/IDU/OneDrive - GTÜ/Desktop/TEZ/poses/arr.npy", allow_pickle=True )
# #print("num of dress poses ",count) ##14008 

# for i, each in enumerate(pose_np):
    
#     if each[0] in df.values:
#         #print(each[0])
#         poselist.append(each)
   
# print("check: ", len(poselist))
 
# np.save(r"C:\Users\IDU\OneDrive - GTÜ\Desktop\TEZ\poses\posesList.npy", poselist, allow_pickle=True)



# if "posesList.npy"  in dir_list:
#     prepare_poses()

# else:
#     poselist = np.load("C:/Users/IDU/OneDrive - GTÜ/Desktop/TEZ/poses/posesList.npy", allow_pickle=True )
"""  
poselist = np.load("C:/Users/IDU/OneDrive - GTÜ/Desktop/TEZ/poses/posesList.npy", allow_pickle=True )   

poselist_df = pd.DataFrame(poselist)   

poselist_df.drop([0,1,2], axis= 1, inplace=True)
"""

# Trying another resnet?

In [28]:
HEIGHT= 100
WIDTH = 135

input_tensor = Input(shape=(HEIGHT, WIDTH, 3))

base_model = ResNet50(
    include_top=False, weights=None, input_tensor=input_tensor,
    input_shape=None)

x= base_model.output
x= GlobalAveragePooling2D()(x)
x= Dense(1024,activation='relu')(x) 
x= Dense(1024,activation='relu')(x) 
model= Model(inputs=base_model.input,outputs=Dense(1)(x))


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import RootMeanSquaredError

# Important, calculate a valid step size for the validation dataset
STEP_SIZE_VALID=test_generator.n//test_generator.batch_size

model.compile(loss = 'mean_squared_error', optimizer='adam', metrics=[RootMeanSquaredError(name="rmse")])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=50, verbose=1, mode='auto',
        restore_best_weights=True)

history = model.fit(train_generator, epochs=100, steps_per_epoch=250, 
                    validation_data = test_generator, callbacks=[monitor],
                    verbose = 1, validation_steps=STEP_SIZE_VALID)

Epoch 1/100
250/250 [==============================] - 3548s 14s/step - loss: 2.9425 - rmse: 1.7154 - val_loss: 0.2320 - val_rmse: 0.4817
Epoch 2/100
175/250 [====================>.........] - ETA: 15:04 - loss: 0.2341 - rmse: 0.4838

In [1]:
model.summary()

NameError: name 'model' is not defined